# Objective <br>
<ol>
    <li>We will first import the IBMD data set to analyze the sentiments (positive or negative)</li>
    <li> We will then run TF and TF-IDF on the sample data set in scikit-learn</li>
    <li>Process the emoticons in the data using the Regular Expression module covered in previous notebook.</li>
    <li>Define stemmers and download nltk stop words corpus</li>
    <li>We now have all the components to define the pipeline with Logistic Regression and evaluate it using Grid Search.</li>
</ol>

In [1]:
from progressbar import ProgressBar
import pandas as pd
import numpy as np
import os
df = pd.DataFrame()

**Load data from file into the dataframe.**

In [2]:
labels = {'pos': 1, 'neg': 0}
basepath = './aclImdb'
#pbar = ProgressBar(max_value=50000)
df = pd.DataFrame()
for s in ('test', 'train'):
    for l in ('pos', 'neg'):
        path = os.path.join(basepath, s, l)
        for file in os.listdir(path):
            # For python2, use 'io.open', for Python3, just us 'open' 
            with open(os.path.join(path, file), 'r', encoding='utf-8') as infile:
                txt = infile.read()
            df = df.append([[txt, labels[l]]], ignore_index=True)
            #pbar.update(1)
df.columns = ['review', 'sentiment']


In [3]:

X = df.loc[:,:].values
X.shape


(50000, 2)

** Save to csv file.**

In [4]:
np.random.seed(0)
df = df.reindex(np.random.permutation(df.index))
df.to_csv('./movie_data.csv', index=False, encoding='utf-8')

df = pd.read_csv('./movie_data.csv')
df.head(5)

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0
3,hi for all the people who have seen this wonde...,1
4,"I recently bought the DVD, forgetting just how...",0


**Initialize vectorizer.**

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(min_df=1)
vectorizer

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

**Count Vectorizer Demo.**

In [1]:
from sklearn.datasets import fetch_20newsgroups # sample data set available in scikit learn 
twenty_train = fetch_20newsgroups(subset='train', shuffle=True)

In [2]:
twenty_train.target_names #prints all the categories
print("\n".join(twenty_train.data[0].split("\n")[:3])) #prints first line of the first data file

From: lerxst@wam.umd.edu (where's my thing)
Subject: WHAT car is this!?
Nntp-Posting-Host: rac3.wam.umd.edu


In [4]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(twenty_train.data) # This will count the frequencies of words in the data set
                                                             # This is eqvivalent to the TF (term frequency part) 
X_train_counts.shape


<1x130107 sparse matrix of type '<class 'numpy.int64'>'
	with 93 stored elements in Compressed Sparse Row format>

** We use the bag of words model. Meaning all the words in the dataset (corpus) are assigned an integer id and their frequency is counted.** <br>
** Each unique word in our corpus will correspond to a descriptive feature.** <br>

** What could go wrong?. Well words that belong to classes like pronouns or articles are found in almost every document (data point), giving more weight to them. But that is not what we would want.** <br>
** To account for this effect we use IDF (Inverse Document Frequency). Meaning we bring down the weight of a word if it appears in many documents(data points).**

In [13]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(11314, 130107)

** Note that the result are stored as a sparse row matrix. **

** Regex to handle emoticons.**

In [6]:
import re
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = re.sub('[\W]+', ' ', text.lower()) +\
        ' '.join(emoticons).replace('-', '')
    return text

In [7]:
df['review'] = df['review'].apply(preprocessor)

** Initialize stemmer. There are other stemmers too like porter, lancaster etc.**

In [8]:
def tokenizer(text):
    return text.split()

In [9]:
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()

def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

In [10]:
from nltk.stem import SnowballStemmer
snowball = SnowballStemmer('english')

def tokenizer_snowball(text):
    return [snowball.stem(word) for word in text.split()]
#df['review'] = df['review'].apply(tokenizer_snowball)

** Download the corpus of stopwords. **

In [11]:
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
stop = stopwords.words('english')

[nltk_data] Downloading package stopwords to D:\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


** Train test split the data. **

In [12]:
X_train = df.loc[:10, 'review'].values # increase the train and test data values. Will take more compute though
y_train = df.loc[:10, 'sentiment'].values # increase the train and test data values. Will take more compute though
X_test = df.loc[11:21, 'review'].values # increase the train and test data values. Will take more compute though
y_test = df.loc[11:21, 'sentiment'].values # increase the train and test data values. Will take more compute though
print(X_train[:5],type(y_train[:5]))

['in 1974 the teenager martha moxley maggie grace moves to the high class area of belle haven greenwich connecticut on the mischief night eve of halloween she was murdered in the backyard of her house and her murder remained unsolved twenty two years later the writer mark fuhrman christopher meloni who is a former la detective that has fallen in disgrace for perjury in o j simpson trial and moved to idaho decides to investigate the case with his partner stephen weeks andrew mitchell with the purpose of writing a book the locals squirm and do not welcome them but with the support of the retired detective steve carroll robert forster that was in charge of the investigation in the 70 s they discover the criminal and a net of power and money to cover the murder murder in greenwich is a good tv movie with the true story of a murder of a fifteen years old girl that was committed by a wealthy teenager whose mother was a kennedy the powerful and rich family used their influence to cover the mu

** We have everything initalized. Building the pipeline. **

In [19]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV

tfidf = TfidfVectorizer(strip_accents=None, lowercase=False, preprocessor=None)

param_grid = [{'vect__ngram_range': [(1, 1)], # specify n gram range
               'vect__stop_words': [stop, None], # use the stop words we downloaded from the nltk corpus Vs. None
               'vect__tokenizer': [tokenizer, tokenizer_porter], # try the two tokenizers we defined
               'clf__penalty': ['l1', 'l2'], # try l1 and l2 regularization
               'clf__C': [1.0, 10.0, 100.0]}, # c values
              {'vect__ngram_range': [(1, 1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer, tokenizer_porter],
               'vect__use_idf':[False],
               'vect__norm':[None],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]},
             ]

lr_tfidf = Pipeline([('vect', tfidf),
                     ('clf', LogisticRegression(random_state=0))])

gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid, scoring='accuracy',
                           cv=5, verbose=1, n_jobs=6) # change n jobs based on your system config

** Run the Logistic regression pipeline we defined above.**

In [20]:
gs_lr_tfidf

GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=False, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
 ...nalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))]),
       fit_params=None, iid=True, n_jobs=6,
       param_grid=[{'vect__ngram_range': [(1, 1)], 'vect__tokenizer': [<function tokenizer at 0x000002785BC3E730>, <function tokenizer_porter at 0x000002785BC3EB70>], 'clf__C': [1.0, 10.0, 100.0], 'clf__penalty': ['l1', 'l2'], 'vect__stop_words': [['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves...kenizer_porter at 0x000002785BC3EB70>], 'clf__C': [1.0, 10.0, 100.0], 'clf__penalty': ['l1', 'l2']}],
       pre_dispatch='2*n_jobs', refit=True, return_tra

In [ ]:
gs_lr_tfidf.fit(X_train, y_train)

Fitting 5 folds for each of 48 candidates, totalling 240 fits


D:\Anaconda\lib\site-packages\sklearn\model_selection\_split.py:605: Warning: The least populated class in y has only 4 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


In [16]:
gs_lr_tfidf.best_params_

{'clf__C': 1.0,
 'clf__penalty': 'l2',
 'vect__ngram_range': (1, 1),
 'vect__stop_words': ['i',
  'me',
  'my',
  'myself',
  'we',
  'our',
  'ours',
  'ourselves',
  'you',
  "you're",
  "you've",
  "you'll",
  "you'd",
  'your',
  'yours',
  'yourself',
  'yourselves',
  'he',
  'him',
  'his',
  'himself',
  'she',
  "she's",
  'her',
  'hers',
  'herself',
  'it',
  "it's",
  'its',
  'itself',
  'they',
  'them',
  'their',
  'theirs',
  'themselves',
  'what',
  'which',
  'who',
  'whom',
  'this',
  'that',
  "that'll",
  'these',
  'those',
  'am',
  'is',
  'are',
  'was',
  'were',
  'be',
  'been',
  'being',
  'have',
  'has',
  'had',
  'having',
  'do',
  'does',
  'did',
  'doing',
  'a',
  'an',
  'the',
  'and',
  'but',
  'if',
  'or',
  'because',
  'as',
  'until',
  'while',
  'of',
  'at',
  'by',
  'for',
  'with',
  'about',
  'against',
  'between',
  'into',
  'through',
  'during',
  'before',
  'after',
  'above',
  'below',
  'to',
  'from',
  'up',
  'do

In [17]:

gs_lr_tfidf.best_score_

0.6363636363636364

In [18]:
clf = gs_lr_tfidf.best_estimator_
clf.score(X_test, y_test)

0.36363636363636365